# DataLab 2: Voetbal & Data Science <a name='begin'></a>

<br>Groepsnummer: 3B
<br>Sprintnummer: 2<br>
<br>    **Busse Heemskerk** 22122877 
<br>    **Isa Dijkstra**                             22119485
<br>    **Johan Heskes**                        22132376
<br>    **Tim Oosterling**                        22078800

## Table of Contents

1. [Importeren van de dataset in Python](#dataset)
2. [Beschrijven van de tabellen](#tabellen)
3. [Bekijken van de data van Heracles Almelo](#hadata)
4. [Eigen functies; Statistieken per seizoen bepalen](#define1)
    1. [get_id](#get_id)
    2. [win_loss](#win_loss)
    3. [statistiek](#statistiek)
    4. [ranglijst](#ranglijst)
5. [Het runnen van de functies](#run1)

## Importeren van de dataset in Python <a name='dataset'></a>

Voordat er gecodeerd wordt, worden de mogelijk nodige libraries voor de opdrachten geïmporteerd

In [ ]:
import sqlite3
import pandas as pd
import numpy as np

Eerst word er een connectie gemaakt met de SQL-database door middel van sqlite3, vervolgens worden de tabellen per stuk geïmporteerd door middel van Pandas.

In [ ]:
connect = sqlite3.connect('database.sqlite')

country = pd.read_sql_query("SELECT * FROM Country", connect)
league = pd.read_sql_query("SELECT * FROM League", connect)
match = pd.read_sql_query("SELECT * FROM Match", connect)
player = pd.read_sql_query("SELECT * FROM Player", connect)
player_attributes = pd.read_sql_query("SELECT * FROM Player_attributes", connect)
team = pd.read_sql_query("SELECT * FROM Team", connect)
team_attributes = pd.read_sql_query("SELECT * FROM Team_attributes", connect)

Vervolgens wordt de optie om de kolommen in te korten uitgezet, daarna worden alle kolommen getoont.

In [ ]:
pd.set_option("display.max_columns", None)
display(country, league, match, player, player_attributes, team, team_attributes)

[Terug naar het begin](#begin)

## Beschrijven van de tabellen <a name='tabellen'></a>

Hier is de beschrijving van elke tabel gegeven, waarbij de naam en de beschrijving te zien zijn. De beschrijving bevat wat er in de tabel te vinden is.

| Tabel naam | Tabel beschrijving 
|:-------------------|:---------------
Country | In deze tabel staan de namen van de landen en de daarbij horende ID's
League | In deze tabel staan de namen van de competities per land samen met landelijke ID
Match | In deze tabel staat alle informatie die met wedstrijden te maken heeft en informatie over het wedden op de wedstrijden
Player | In deze tabel staat info over alle spelers, hun fifa ID, wanneer ze jarig zijn, hun lengte (in cm) en hun gewicht (in lbs) 
Player_attributes | In deze tabel staan de statistieken en rating van elke speler in de database 
Team | In deze tabel staat de id's van de club, de volledige club naam en de afkorting van de club
Team_attributes | In deze tabel zijn de statistieken van de teams in de database vermeld. zowel offensieve als defensieve statestieken zijn hierbij inbegrepen

[Terug naar het begin](#begin)

## Bekijken van de data van Heracles Almelo <a name='hadata'></a>

Eerst word er een cell gerunt om de team ID te vinden van onze club: Heracles Almelo

In [ ]:
team[team["team_long_name"] == "Heracles Almelo"]

De gegevens van het gekozen team staan getoond in de uitkomst van de onderstaande cell.

In [ ]:
team_id = team[team['team_long_name'] == 'Heracles Almelo'].iloc[0,1]
print('Team ID:', team_id)
print('Name:', team[team['team_long_name'] == 'Heracles Almelo'].iloc[0,3])
print('Short name:', team[team['team_long_name'] == 'Heracles Almelo'].iloc[0,4])
print('Country:', 'Netherlands')
print('League id:', country[country['name'] == 'Netherlands'].iloc[0,0])
print("League name:", league[league["id"] == 13274].iloc[0,2])

De gegevens van elke wedstrijd die het gekozen team heeft gespeeld per seizoen is hieronder getoond.

In [ ]:
display(match[(match['home_team_api_id'] == team_id) | (match['away_team_api_id'] == team_id)])

[Terug naar het begin](#begin)

## Eigen functies; Statistieken per seizoen bepalen <a name='define1'></a>

Hieronder zijn 4 functies gedefineerd:

    id_zoeker
    punten
    statistiek
    ranglijst
De uitleg van alle functies staat omschreven in de docstrings

### id_zoeker <a name='id_zoeker'></a>

In [ ]:
def id_zoeker(club):
    """
    Deze functie haalt het ID-nummer van de gegeven club op
    
    Parameters
    ----------
    club : str 
        De volledige naam van de club, waarvan je het ID wilt weten
    
    Returns
    ----------
    id : integer
        Het ID-nummer van de gekozen club
        
    """
    
    #Filteren
    id = team[team['team_long_name'] == club].iloc[0,1]
    return id

[Terug naar het begin](#begin)

### punten <a name='punten'></a>

In [ ]:
def punten(club, seizoen):
    """
    Deze functie maakt, op basis van een club en een seizoen, 2 kolommen aan. Eén over het aantal punten en één over 
    of de wedstrijd thuis of uit was voor het gekozen team.
    
    Parameters
    ----------
    club : str 
        De volledige naam van de club, waarvan je wilt weten hoeveel er gescoord is
    
    seizoen : str
        Het speel seizoen waar je de data van wilt weten
    
    Returns
    ----------
    wed : Pandas DataFrame
        Een dataframe gebaseerd op match, waarbij alleen de data van de aangegeven club en seizoen wordt getoond
    
    """
    
    #Aanmaken van een nieuwe gefilterde dataset en 2 lege lists
    wed = match[((match['home_team_api_id'] == id_zoeker(club)) | (match['away_team_api_id'] == id_zoeker(club))) & (match['season'] == seizoen)]
    punten_ = []
    tu = []
    
    #Beginnen van een loop om de lijsten te vullen
    loop = 0
    for i in list(wed['home_team_api_id']):
        #If-statement voor wanneer het team thuis of uit speelt
        if list(wed['home_team_api_id'])[loop] == id_zoeker(club):
            #If-statement voor wanneer het team meer of minder doelpunten heeft gescoord om te kijken naar winst, gelijk en verlies
            if list(wed['home_team_goal'])[loop] > list(wed['away_team_goal'])[loop]:
                punten_.append(3)
                tu.append('thuis')
            elif list(wed['home_team_goal'])[loop] == list(wed['away_team_goal'])[loop]:
                punten_.append(1)
                tu.append('thuis')
            else:
                punten_.append(0)
                tu.append('thuis')
        else:
            #If-statement voor wanneer het team meer of minder doelpunten heeft gescoord om te kijken naar winst, gelijk en verlies
            if list(wed['away_team_goal'])[loop] > list(wed['home_team_goal'])[loop]:
                punten_.append(3)
                tu.append('uit')
            elif list(wed['away_team_goal'])[loop] == list(wed['home_team_goal'])[loop]:
                punten_.append(1)
                tu.append('uit')
            else:
                punten_.append(0)
                tu.append('uit')
        loop += 1
        
    #Aanmaken van copy en 2 nieuwe kolommen
    wed = wed.copy()
    wed.loc[:, 'punten'] = punten_
    wed.loc[:, 'thuis / uit'] = tu
    return wed

[Terug naar het begin](#begin)

### statistiek <a name='statistiek'></a>

In [ ]:
def statistiek(club, seizoen):
    """
    Deze functie laat de statistieken zien over de gegeven club in het gegeven seizoen. 
    
    Parameters
    ----------
    club : str 
        De volledige naam van de club, waarvan je de statistieken wilt weten
    
    seizoen : str
        Het speel seizoen waar je de data van wilt weten
    
    Returns
    ----------
    df : Pandas DataFrame
        Een dataframe waarin de naam, het aantal wedstrijden, het aantal punten, het gemiddelde aantal punten per wedstrijd en de winst percentages staan
    
    """
    #Aanmaken van 2 aparte dataframes voor percentages
    t = punten(club, seizoen)[punten(club, seizoen)['thuis / uit'] == 'thuis']
    u = punten(club, seizoen)[punten(club, seizoen)['thuis / uit'] == 'uit']
    
    #Maken van een list met alle benodigde data voor het dataframe 
    stats = [[club,
            seizoen,
            len(punten(club, seizoen)),
            sum(punten(club, seizoen)['punten']),
            round(sum(punten(club, seizoen)['punten'] / len(punten(club, seizoen)['punten'])), 2),
            round(t[t['punten'] == 3]['punten'].count() / t['punten'].count() * 100, 2),
            round(t[t['punten'] == 1]['punten'].count() / t['punten'].count() * 100, 2),
            round(t[t['punten'] == 0]['punten'].count() / t['punten'].count() * 100, 2),
            round(u[u['punten'] == 3]['punten'].count() / u['punten'].count() * 100, 2),
            round(u[u['punten'] == 1]['punten'].count() / u['punten'].count() * 100, 2),
            round(u[u['punten'] == 0]['punten'].count() / u['punten'].count() * 100, 2)]]
    
    #Maken van het dataframe
    df = pd.DataFrame(stats, columns =['team',
                                       'seizoen',
                                       'wedstrijden', 
                                       'punten',
                                       'gemiddeld_punten',
                                       'thuis_win%',
                                       'thuis_gelijk%',
                                       'thuis_verlies%',
                                       'uit_win%',
                                       'uit_gelijk%',
                                       'uit_verlies%'])
       
    return df

[Terug naar het begin](#begin)

### ranglijst <a name='ranglijst'></a>

In [ ]:
load_screen = ["Welcome to this journey!..............................................",    
                       "Preparing to load dataset.............................................",    
                       "Checking for missing files...............................................",    
                       "Initializing data structures................................................",    
                       "Setting up parameters.....................................................",    
                       "Loading feature vectors................................................",    
                       "Applying pre-processing steps......................................",    
                       "Filtering out invalid samples.............................................",    
                       "Compiling statistics.......................................................",    
                       "Normalizing data...........................................................",    
                       "Building training and test sets..........................................",    
                       "Cross-validating models.................................................",   
                       "Fine-tuning hyperparameters..........................................",    
                       "Evaluating performance metrics....................................",    
                       "The dataset is almost loaded!........................................"]

def ranglijst(league_naam,seizoen):
    """
    Deze functie laat de ranglijst zien over de gegeven league in het gegeven seizoen. 
    
    Parameters
    ----------
    league_naam : str 
        De naam van de league waarvan je de ranglijst wilt bepalen
    
    seizoen : str
        Het speel seizoen waar je de data van wilt weten
    
    Returns
    ----------
    df : Pandas DataFrame
        Een dataframe waarin de data van het gegeven seizoen staat op volgorde van de behaalde punten
        
    """
    
    #Filteren voor het maken van een lijst met alle team namen in een league
    league_id = league[league['name']== league_naam].iloc[0,1]
    seizoen_filter = match[match['season'] == seizoen]
    team_ids = set(list(seizoen_filter[seizoen_filter['league_id'] == league_id]['home_team_api_id']))
    team_ids = list(team_ids)
    
    #Maken van een lijst met alle team namen in een league
    team_names = []
    for i in team_ids:
        team_names.append(team[team['team_api_id'] == i].iloc[0,3])
    
    #Aanmaken dataframe voor gebruik    
    df = statistiek(team_names[0],seizoen)

    #Loopen over de list met team namen om het dataframe vol te zetten
    print('stats season', seizoen, 'from', league_naam)
    for i in range(1, len(team_names)):
        df = pd.concat([df, statistiek(team_names[i], seizoen)])
        print(str(round(100/(len(team_names)+1)*i,2))+'%', load_screen[(i-1)%15], end = '\r')
    print('100.0% This adventure was brought to you by Group 3B.........')
    
    #Sorteren van het dataframe op basis van behaalde punten
    df = df.sort_values(by='punten', ascending = False).set_index('team')
    
    return df 

[Terug naar het begin](#begin)

## Het runnen van de functies <a name='run1'></a>
Hieronder worden de functies statistiek en ranglijst uitgevoerd:

In [ ]:
statistiek('Heracles Almelo', '2009/2010')

In [ ]:
ranglijst('Netherlands Eredivisie', '2009/2010')

#### Einde Sprint 2

[Terug naar het begin](#begin)